# Basic SQLAlchemy ORM manipulations

In [ ]:
import json
from os import environ

from sqlalchemy import create_engine, text, func
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, aliased, selectinload, joinedload
from sqlalchemy import Column, ForeignKey, Integer, String, Float
from sqlalchemy.orm import relationship, backref
from sqlalchemy.orm.exc import MultipleResultsFound
from sqlalchemy.sql import exists

# Setup

In [ ]:
host = environ.get('PG_SERVER', 'sql-fabulous')
db = environ.get('PG_DATABASE', 'condesa')
user = environ.get('PG_UID', 'sa')
pw = environ.get('POSTGRES_PASSWORD', 'pwd')

con_str = f'{user}:{pw}@{host}/{db}'

# 'echo' emits generated sql
engine = create_engine(f"postgresql+psycopg2://{con_str}", echo=True)

# Define the catalog of classes

In [ ]:
Base = declarative_base()


class Survey(Base):
    __tablename__ = 'directional_survey'
    
    id = Column(Integer, autoincrement=True, primary_key=True)
    api = Column(String(32), nullable=True)
    wkid = Column(String(32), nullable=True)
    fips = Column(String(4), nullable=True)
    status_code = Column(String(1), nullable=False)

    def __repr__(self):
        return f"Survey(id={self.id}, api={self.api}, status={self.status_code})"

class SurveyReport(Base):
    __tablename__ = 'survey_report'
    
    id = Column(Integer, autoincrement=True, primary_key=True)
    directional_survey_id = Column(Integer, ForeignKey('directional_survey.id'), nullable=False)
    azimuth = Column(Float, nullable=True)
    md = Column(Float, nullable=True)
    inclination = Column(Float, nullable=True)
    status_code = Column(String(1), nullable=False)

    # Relationship
    survey = relationship(Survey, backref=backref('stations', uselist=True))
    #survey = relationship(Survey, back_populates='stations')

    def __repr__(self):
        return f"SurveyReport(id={self.id}, FK={self.directional_survey_id}, status={self.status_code})"
    

# For SQL Core: set mapped tables to local vars so that full SQL metadata is available.
surveys = Survey.__table__
points = SurveyReport.__table__

# Inspect underlying metadata

In [ ]:
SurveyReport.__table__

# Create Schema

In [ ]:
Base.metadata.drop_all(engine, checkfirst=True)
Base.metadata.create_all(engine, checkfirst=False)

# Create session - interface to the DB

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

# Create Beans

In [ ]:
try:
    # Automatically creates a constructor accepting kwargs
    survey1 = Survey(api='API1', wkid='WKID1', fips = '0001', status_code='C')

    survey_report1 = SurveyReport(directional_survey_id=survey1.id, azimuth=1, md=1, inclination=1, status_code='C')
    survey_report2 = SurveyReport(directional_survey_id=survey1.id, azimuth=2, md=2, inclination=1, status_code='C')
    survey_report3 = SurveyReport(directional_survey_id=survey1.id, azimuth=3, md=3, inclination=1, status_code='C')

    survey1.stations.append(survey_report1)
    survey1.stations.append(survey_report2)
    survey1.stations.append(survey_report3)

    session.add(survey1)
    
    survey2 = Survey(api='API2', wkid='WKID2', fips = '0002', status_code='C')

    survey_report2b = SurveyReport(directional_survey_id=survey2.id, azimuth=2, md=2, inclination=2, status_code='N')
    survey_report3b = SurveyReport(directional_survey_id=survey2.id, azimuth=3, md=3, inclination=2, status_code='N')
    survey_report4b = SurveyReport(directional_survey_id=survey2.id, azimuth=4, md=4, inclination=2, status_code='N')

    survey2.stations = [survey_report2b, survey_report3b, survey_report4b]

    session.add(survey2)
    
    survey3 = Survey(api='API3', wkid='WKID3', fips = '0003', status_code='N')

    survey_report2c = SurveyReport(directional_survey_id=survey3.id, azimuth=2, md=2, inclination=2, status_code='N')
    survey_report3c = SurveyReport(directional_survey_id=survey3.id, azimuth=3, md=3, inclination=2, status_code='N')
    survey_report4c = SurveyReport(directional_survey_id=survey3.id, azimuth=4, md=4, inclination=2, status_code='N')

    survey3.stations.extend([survey_report2c, survey_report3c, survey_report4c])
    
    session.add_all([survey3])

    survey4 = Survey(id=4, api='API4', wkid='0004', status_code='C')

    session.add(survey4)
    
    session.commit()

except SQLAlchemyError as e:
    print(e)
    session.rollback()

# Update

In [ ]:
survey_mod = session.query(Survey).filter_by(api = 'API4').first()
survey_mod.wkid = 'WKID4'
survey_mod.fips = '0004'

try:
    session.add_all([survey_mod])
    session.commit()
except SQLAlchemyError as e:
    session.rollback()

# Querying

In [ ]:
for i in session.query(Survey).order_by(Survey.id):
    print(f"{i.id}, {i.api}")

In [ ]:
for id, api in session.query(Survey.id, Survey.api):
    print(f"{id}, {api}")

In [ ]:
for row in session.query(Survey, Survey.id).all():
    print(f"{row.Survey}, {row.Survey.stations}, {row.id}")

In [ ]:
for row in session.query(Survey, Survey.id.label('survey_id')).all(): # field aliasing
    print(f"{row.Survey}, {row.survey_id}")

In [ ]:
# table aliasing
DirectionalSurvey = aliased(Survey, name='d_survey')
for row in session.query(DirectionalSurvey, DirectionalSurvey.id):
    print(f"{row.d_survey}, {row.id}")

In [ ]:
# Can use python slicer to limit results.
for row in session.query(Survey).order_by(Survey.id)[-1::-1]:
    print(f"{row.id}")

In [ ]:
# Where clause
for s in session.query(Survey).filter(Survey.api == "API4"):
    print(f"{s}")

In [ ]:
# Where clause AND
# ilike applies a lower to both sides
for s in (session.
          query(Survey).
          filter(Survey.api.startswith("API")).
          filter(Survey.wkid.like("WK%")).
          filter(Survey.wkid.ilike("wk%")).
          filter(Survey.status_code == "N")
         ):
    print(f"{s}")

In [ ]:
# IN/NOT IN Sub-query
for s in (session.
          query(Survey).
          filter(~Survey.id.in_(
              session.
              query(SurveyReport.directional_survey_id)
            ))
         ):
    print(f"{s}")

In [ ]:
for sr in session.query(SurveyReport):
    print(f"{sr}")

# Other query operators
- "==" is the same as "is_"
- "!=" is the same as "isnot"
- Compound filters are the same as "and_"
- "or_" can also be used
- ""== None" is the same as "IS / IS NOT NULL"
- contains()

# Result Operators all, first, one

In [ ]:
for r in session.query(Survey).all(): # returns list
    print(r)

In [ ]:
r = session.query(Survey).first() # returns single tuple
print(r)

In [ ]:
try:
    # Fails if more than one exists 
    # "one_or_none()" returns None instead of failing.
    r = session.query(Survey).one() 
except MultipleResultsFound as e:
    print(e)

# Bind Parameters

In [ ]:
for s in (session.
          query(Survey).
          filter(text("id < :rec and api = :api")).
          params(rec=4, api='API2')):
    print(s)

# Literal SQL

In [ ]:
s = (session.
     query(Survey).
     from_statement(
        text("SELECT id, api FROM directional_survey where api = :api")).
     params(api='API1')
     .first()
    )

s

# COUNT()

In [ ]:
g = (session.
     query(func.count(Survey.status_code), Survey.status_code).
     group_by(Survey.status_code).
     all()
    )
g

In [ ]:
session.query(func.count(Survey.id)).scalar() # count(*)

# Navigating Relations

In [ ]:
r = session.query(Survey).first() # returns single tuple
print(r.stations[0].survey.api)

# Joins

In [ ]:
# Old SQL cross-join style
for s, sr in (session.
              query(Survey, SurveyReport).
              filter(Survey.id == SurveyReport.directional_survey_id).
              filter(Survey.status_code.in_(['N']))
             ):
    print(f"{s}, {sr}")

In [ ]:
# Standard style
# join can be explicit: 
# join(SurveyReport, Survey.ID == SurveyReport.DirectionalSurveyId)
# outerjoin() also available
# Use table aliasing to join to the same table more than once.
for row in (session.
            query(Survey).
            join(SurveyReport).
            filter(Survey.status_code.in_(['N']))
           ):
    print(f"{row}, {row.stations}")

# Sub-queries

In [ ]:
# Build named sub-query
stmt = (session.
        query(SurveyReport.directional_survey_id, 
              func.count('*').
              label('report_count')).
        group_by(SurveyReport.directional_survey_id).
        subquery()
       )

# Join to parent table
for u, count in (session.
                 query(Survey, stmt.c.report_count).
                 outerjoin(stmt, Survey.id == stmt.c.directional_survey_id).
                 order_by(Survey.id)
                ):
    print(u, count)

In [ ]:
# Exists/Not Exists use "~"
# Use has() for many to 1 relations
for s in (session.
          query(Survey).
          filter(
              Survey.stations.
              any(SurveyReport.status_code.in_(['N', 'C'])))
         ): # or any()
    print(s)

# Optimizations

In [ ]:
# Loads all related objects eagerly
s1 = (session.
      query(Survey).
      options(selectinload(Survey.stations)).
      filter(Survey.api == 'API1').one()
     )

s1

In [ ]:
# Loads all related objects eagerly
jack = (session.
        query(Survey).
        options(joinedload(Survey.stations)).
        filter_by(api='API1').
        one()
       )
jack

# Deletes

In [ ]:
sr1 = (session.
       query(SurveyReport).
       filter(SurveyReport.id == 9).
       one()
      )

try:
    session.delete(sr1)
    session.commit()
except SqlAlchemyException as e:
    session.rollback()